# Best Player and Game Leaders
This notebook demonstrates two analytics utilities:
- Compute the match 'Best Player' from goals and assists.
- Compute per-team 'Game Leaders' for Goals, Assists, and Cards.

The logic mirrors the frontend implementation (match.js) and is designed to work with common AllSports event shapes.

In [1]:
from typing import Any, Dict, List, Optional

def _coalesce(*vals):
    for v in vals:
        if v is not None and v != '':
            return v
    return ''

def compute_best_player_from_event(ev: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    players: Dict[str, Dict[str, int]] = {}
    def add_goal(name: str):
        if not name: return
        players.setdefault(name, { 'goals': 0, 'assists': 0 })
        players[name]['goals'] += 1
    def add_assist(name: str):
        if not name: return
        players.setdefault(name, { 'goals': 0, 'assists': 0 })
        players[name]['assists'] += 1

    goalscorers = ev.get('goalscorers') or ev.get('goals') or ev.get('goalscorer') or []
    if isinstance(goalscorers, list):
        for g in goalscorers:
            hs = _coalesce(g.get('home_scorer'), g.get('home_scorer_name'), g.get('home_scorer_fullname'))
            ha = _coalesce(g.get('home_assist'), g.get('home_assist_name'))
            as_ = _coalesce(g.get('away_scorer'), g.get('away_scorer_name'), g.get('away_scorer_fullname'))
            aa = _coalesce(g.get('away_assist'), g.get('away_assist_name'), g.get('away_assist_fullname'))
            add_goal(hs); add_goal(as_)
            add_assist(ha); add_assist(aa)

    best = None
    max_score = -1
    for name, st in players.items():
        score = st.get('goals', 0)*3 + st.get('assists', 0)*1
        if score > max_score:
            max_score = score
            best = {
                'name': name,
                'score': score,
                'reason': f"{st.get('goals',0)} goals, {st.get('assists',0)} assists"
            }
    return best

def compute_team_leaders(ev: Dict[str, Any]) -> Dict[str, Any]:
    home_name = _coalesce(ev.get('event_home_team'), ev.get('strHomeTeam'), ev.get('home_team'), 'Home')
    away_name = _coalesce(ev.get('event_away_team'), ev.get('strAwayTeam'), ev.get('away_team'), 'Away')

    def make_player(side: str, name: str):
        return { 'name': name, 'side': side, 'goals': 0, 'assists': 0, 'yc': 0, 'rc': 0 }
    maps = { 'home': {}, 'away': {} }
    def get_or(side: str, name: str):
        if not name: return None
        if name not in maps[side]: maps[side][name] = make_player(side, name)
        return maps[side][name]

    # goals + assists
    goalscorers = ev.get('goalscorers') or ev.get('goals') or ev.get('goalscorer') or []
    if isinstance(goalscorers, list):
        for g in goalscorers:
            hs = _coalesce(g.get('home_scorer'), g.get('home_scorer_name'), g.get('home_scorer_fullname'))
            ha = _coalesce(g.get('home_assist'), g.get('home_assist_name'))
            as_ = _coalesce(g.get('away_scorer'), g.get('away_scorer_name'), g.get('away_scorer_fullname'))
            aa = _coalesce(g.get('away_assist'), g.get('away_assist_name'), g.get('away_assist_fullname'))
            if hs: get_or('home', hs)['goals'] += 1
            if as_: get_or('away', as_)['goals'] += 1
            if ha: get_or('home', ha)['assists'] += 1
            if aa: get_or('away', aa)['assists'] += 1

    # cards
    cards = ev.get('cards') or ev.get('bookings') or ev.get('events_cards') or []
    if isinstance(cards, list):
        for c in cards:
            is_home = bool(c.get('home_fault') or c.get('home_player') or c.get('home_scorer'))
            is_away = bool(c.get('away_fault') or c.get('away_player') or c.get('away_scorer'))
            name = _coalesce(c.get('home_fault'), c.get('away_fault'), c.get('player'), c.get('player_name'))
            typ = (c.get('card') or c.get('type') or '').lower()
            if not name: continue
            side = 'home' if is_home else ('away' if is_away else None)
            if not side: continue
            rec = get_or(side, name)
            if 'red' in typ: rec['rc'] += 1
            elif 'yellow' in typ: rec['yc'] += 1

    def pick_leader(side: str, key: str):
        arr = list(maps[side].values())
        if not arr: return None
        if key == 'cards':
            arr.sort(key=lambda o: (o['rc'], o['yc'], o['goals'] + o['assists']), reverse=True)
        else:
            arr.sort(key=lambda o: o.get(key, 0), reverse=True)
        top = arr[0]
        if key == 'goals' and top['goals'] <= 0: return None
        if key == 'assists' and top['assists'] <= 0: return None
        if key == 'cards' and top['rc'] <= 0 and top['yc'] <= 0: return None
        return top

    leaders = {
        'homeTeamName': home_name,
        'awayTeamName': away_name,
        'home': {
            'goals': pick_leader('home', 'goals'),
            'assists': pick_leader('home', 'assists'),
            'cards': pick_leader('home', 'cards'),
        },
        'away': {
            'goals': pick_leader('away', 'goals'),
            'assists': pick_leader('away', 'assists'),
            'cards': pick_leader('away', 'cards'),
        },
    }
    return leaders

In [2]:
# Sample event payload (simplified)
event = {
    'event_home_team': 'Rayo Vallecano',
    'event_away_team': 'Celta Vigo',
    'goalscorers': [
        {'time': '18', 'home_scorer': 'J. de Frutos'},
        {'time': '44', 'away_assist': 'H. Alvarez'}
    ],
    'cards': [
        {'time': '72', 'home_fault': 'I. Palazon', 'card': 'yellow card'}
    ]
}

bp = compute_best_player_from_event(event)
leaders = compute_team_leaders(event)
bp, leaders

({'name': 'J. de Frutos', 'score': 3, 'reason': '1 goals, 0 assists'},
 {'homeTeamName': 'Rayo Vallecano',
  'awayTeamName': 'Celta Vigo',
  'home': {'goals': {'name': 'J. de Frutos',
    'side': 'home',
    'goals': 1,
    'assists': 0,
    'yc': 0,
    'rc': 0},
   'assists': None,
   'cards': {'name': 'I. Palazon',
    'side': 'home',
    'goals': 0,
    'assists': 0,
    'yc': 1,
    'rc': 0}},
  'away': {'goals': None,
   'assists': {'name': 'H. Alvarez',
    'side': 'away',
    'goals': 0,
    'assists': 1,
    'yc': 0,
    'rc': 0},
   'cards': None}})

In [3]:
# Pretty print leaders side-by-side
def fmt_player(p):
    if not p: return '—'
    parts = []
    if p.get('goals'): parts.append(f"{p['goals']} GLS")
    if p.get('assists'): parts.append(f"{p['assists']} AST")
    if p.get('rc') or p.get('yc'): parts.append(f"{p.get('rc',0)} RC {p.get('yc',0)} YC")
    stat = ' '.join(parts) if parts else ''
    return f"{p['name']}" + (f" — {stat}" if stat else '')

def print_leaders(leaders):
    print(f"{leaders['homeTeamName']} vs {leaders['awayTeamName']}")
    rows = [
        ('GOALS', leaders['home']['goals'], leaders['away']['goals']),
        ('ASSISTS', leaders['home']['assists'], leaders['away']['assists']),
        ('CARDS', leaders['home']['cards'], leaders['away']['cards']),
    ]
    for title, hp, ap in rows:
        print(f"{title:8} | {fmt_player(hp):30} | {fmt_player(ap):30}")

print('Best Player:', bp)
print_leaders(leaders)

Best Player: {'name': 'J. de Frutos', 'score': 3, 'reason': '1 goals, 0 assists'}
Rayo Vallecano vs Celta Vigo
GOALS    | J. de Frutos — 1 GLS           | —                             
ASSISTS  | —                              | H. Alvarez — 1 AST            
CARDS    | I. Palazon — 0 RC 1 YC         | —                             


## Notes
- Best Player scoring is a simple heuristic: 3 points per goal, 1 per assist.
- Team leaders choose top contributors for each category; cards prioritize red over yellow.
- You can paste a real event payload (provider JSON) into the sample cell to test against live-like data.
- If your environment can import project modules, you can adapt these functions to reuse server-side code.